In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import TensorBoard
from tensorflow import keras
import tensorflow as tf
from scipy import stats
import pandas as pd
import datetime, os
import numpy as np

Using TensorFlow backend.


In [ ]:
# Import the preprocessed file we made
print("File name: ") 
file_name = input()

dataFrame = pd.read_csv(r'data/p/' + file_name + '.csv')
dataFrame.head()

File name: 


In [ ]:
num_features = dataFrame.shape[1] - 1
print("We have " + str(num_features) + " features") 

In [ ]:
target_name = input()
y = dataFrame.pop(target_name) - 1
X = dataFrame

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)


This model will be used for the NSP classification

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, input_dim=num_features, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='softmax')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Tensorboard stuff
log_dir = os.path.join(
    "logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir, histogram_freq=1)

# Train the model
model.fit(x=X_train,
          y=y_train,
          epochs=100,
          validation_data=(X_val, y_val),
          callbacks=[tensorboard_callback])